In [1]:
import os
import numpy as np
from absl import flags, app
from model import MLQP
from utils import dataloader, set_seed
from tqdm import trange
import pandas as pd

lr = 1e-1
num_hidden = 32
num_epoch = 2000
name = "mlqp"


In [2]:
train_data = dataloader("train")
test_data = dataloader("test")
mlqp = MLQP(1, 2, num_hidden, 1)
train_loss, train_acc, test_loss, test_acc = [], [], [], []

with trange(1, num_epoch + 1, desc='Training', ncols=0) as pbar:
    for epoch in pbar:
        #train
        preds = []
        for cor, y in train_data:
            pred = mlqp.forward(cor)
            mlqp.backward(y)
            mlqp.step(lr)
            preds.append(pred)
        preds = np.squeeze(np.array(preds))
        loss = (0.5 * (train_data[:, 1] - preds) ** 2).sum()
        acc = ((preds > 0.5) == train_data[:, 1]).mean()
        pbar.set_postfix(loss='%.4f' % loss, acc='%.4f' % acc)
        train_loss.append(loss)
        train_acc.append(acc)

        # test
        if epoch % 50 == 0:
            preds = []
            for cor, y in train_data:
                pred = mlqp.forward(cor)
                preds.append(pred)
            preds = np.squeeze(np.array(preds))
            loss = (0.5 * (train_data[:, 1] - preds) ** 2).sum()
            acc = ((preds > 0.5) == train_data[:, 1]).mean()
            pbar.set_postfix(loss='%.4f' % loss, acc='%.4f' % acc)
            train_loss.append(loss)
            train_acc.append(acc)


/Users/liyang/Documents/GitRepos/Minmax-MLQP/utils.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(res)
Training: 100% 2000/2000 [00:31<00:00, 64.07it/s, acc=0.9467, loss=6.0508] 
